河野君の見付けた d-surface のモジュールの可視化をこころみる

Clustice を使えば、グラフを氷の構造にできる、はず。


In [1]:
from logging import getLogger, INFO, DEBUG
import numpy as np
import networkx as nx
import py3Dmol

import genice_core
from clustice.serialize import serialize
from clustice.geometry import make_layout
from clustice import gromacs, yaplot
from clustice.water import tip4p, spce
from clustice import graph
from cycless import cycles

logger = getLogger()
logger.setLevel(INFO)

# O-O distance
L = 0.27

g = nx.Graph(
    [
        [0, 20],
        [0, 23],
        [1, 2],
        # [2, 3],
        # [3, 4],
        [3, 22],
        [3, 18],
        [4, 1],
        [5, 6],
        # [6, 7],
        # [7, 8],
        [19, 6],
        [8, 21],
        [8, 5],
        [9, 10],
        [10, 11],
        [11, 12],
        [12, 9],
        [13, 14],
        [14, 15],
        [15, 16],
        [16, 13],
        [1, 17],
        [17, 5],
        [17, 20],
        [17, 23],
        [20, 19],
        # [19, 8],
        [7, 19],
        [19, 9],
        [20, 18],
        [18, 4],
        [18, 10],
        # [6, 21],
        [7, 21],
        [21, 13],
        [21, 23],
        # [23, 7],
        [23, 22],
        [22, 2],
        [22, 14],
        [12, 24],
        [24, 16],
        [11, 25],
        [25, 15],
        [25, 0],
        [24, 0],
        [3, 25],
        [7, 24],
        [6, 26],
        [7, 26],
        [8, 26],
        [2, 27],
        [3, 27],
        [4, 27],
        [9, 28],
        [10, 29],
        [11, 30],
        [12, 31],
        [28, 29],
        [29, 30],
        [30, 31],
        [31, 28],
        [13, 32],
        [14, 33],
        [15, 34],
        [16, 35],
        [32, 33],
        [33, 34],
        [34, 35],
        [35, 32],
    ]
)

if "pos" in g.nodes[0]:
    # extract the embedded coords in g
    mol_positions = np.array([g.nodes[v]["pos"] for v in g])
else:
    # estimate of the positions of the nodes
    mol_positions = make_layout(g, edge_length=L)


yap = yaplot.render(g, mol_positions)
with open("dmodule.yap", "w") as f:
    f.write(yap)


# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg = genice_core.ice_graph(
    g, vertexPositions=mol_positions, dipoleOptimizationCycles=1000
)
# dg = ice_graph(g)

# get the unique id for the graph
# id = serialize(dg)
# print(id)

# put water molecules
gro = gromacs.render(
    dg,
    mol_positions,
    water_model=spce,
    cell_matrix=np.array([[30.0, 0.0, 0.0], [0.0, 30.0, 0.0], [0.0, 0.0, 30.0]]),
    shift=np.array([0.5, 0.5, 0.5]),
)
with open("dmodule.gro", "w") as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})  #: {"radius": 0.75}})
view.addUnitCell()
view.zoomTo()
view.show()
view.png()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

[FINDSYM](https://stokes.byu.edu/iso/findsym.php)で原子座標から空間群を推定できる。

この構造は Imma と推定された。

上のモジュールを単位胞に 4 つ並べると完成すると思われる。

並べてみるためには、まず上で作った座標を主軸変換する必要がある。


In [ ]:
# 慣性テンソルを作る。
# 質点の質量は等しいと仮定する。

I = np.zeros([3, 3])
for x in range(3):
    for y in range(3):
        I[y, x] = mol_positions[:, x] @ mol_positions[:, y]

# 固有行列
I_p, P = np.linalg.eig(I)

# 上下反転
R = np.diag([1, 1, -1])

aligned = mol_positions @ P @ R

# 慣性テンソルが対角化されたことを確認
# Io = np.zeros([3,3])
# for x in range(3):
#     for y in range(3):
#         Io[y,x] = aligned[:,x] @ aligned[:,y]
# print(Io)


yap = yaplot.render(g, aligned)
with open("d-aligned.yap", "w") as f:
    f.write(yap)

In [59]:
# てきとうにセルを定義する。
# cell = np.diag([2.2, 1.7, 2.8])
cell = np.diag([2.1, 2.1 * 0.95, 2.1 * 2**0.5 * 1.15])
celli = np.linalg.inv(cell)

# 1 unitを配置する
relp = aligned @ celli + np.array([0, 1 / 4, 1 / 8])
atomd = dict()
for i in range(relp.shape[0]):
    atomd[f"O{i}"] = relp[i]

# cifを利用して、残りの原子の位置を特定する。
# I m m a
symops = """
      x,            y,            z
     -x,            y,            z
      x,          1/2-y,          z
      x,          1/2+y,         -z
 
     -x,           -y,           -z
      x,           -y,           -z
     -x,          1/2+y,         -z
     -x,          1/2-y,          z

      x+1/2,            y+1/2,            z+1/2
     -x+1/2,            y+1/2,            z+1/2
      x+1/2,          1/2-y+1/2,          z+1/2
      x+1/2,          1/2+y+1/2,         -z+1/2
 
     -x+1/2,           -y+1/2,           -z+1/2
      x+1/2,           -y+1/2,           -z+1/2
     -x+1/2,          1/2+y+1/2,         -z+1/2
     -x+1/2,          1/2-y+1/2,          z+1/2

    """.translate(
    {ord(","): ""}
)
#   x,            y,            z
#  -x,            y,            z
#   x,          1/2-y,          z
#   x,          1/2+y,         -z

#  -x,           -y,           -z
#   x,           -y,           -z
#  -x,          1/2+y,         -z
#  -x,          1/2-y,          z

from genice2 import CIF

sops = CIF.symmetry_operators(symops)

atoms = CIF.fullatoms(atomd, sops)
atoms = np.array([x[1] for x in atoms])

import yaplotlib as yap

s = yap.Size(0.1)
for i, atom in enumerate(atoms @ cell):
    s += yap.Circle(atom)
    s += yap.Text(atom, f"{i}")
with open("d-unitcell.yap", "w") as f:
    f.write(s)

In [61]:
# だいたい形はできた。ずれは無理矢理解消する。

import pairlist as pl

id = [
    [31, 140],
    [30, 141],
    [29, 142],
    [28, 143],
    [34, 137],
    [35, 136],
    [33, 138],
    [32, 139],
    [65, 106],
    [64, 107],
    [66, 105],
    [67, 104],
    [69, 102],
    [68, 103],
    [70, 101],
    [71, 100],
]

elim = set()
for i, j in id:
    d = atoms[j] - atoms[i]
    d -= np.floor(d + 0.5)
    c = atoms[i] + d * 0.5
    atoms[i] = c
    elim.add(j)

remain = set(range(atoms.shape[0])) - elim
atomr = atoms[list(remain)]
atomr -= np.floor(atomr)

s = yap.Size(0.1)
s += yap.Color(2)

for atom in atomr:
    s += yap.Circle(atom @ cell)

s += yap.Color(3)
for i, j in pl.pairs_iter(atomr, 0.34, cell, distance=False):
    d = atomr[j] - atomr[i]
    d -= np.floor(d + 0.5)
    s += yap.Line(atomr[i] @ cell, (atomr[i] + d) @ cell)

with open("d-unitcell.yap", "w") as f:
    f.write(s)

In [ ]:
# 最後に、これをグラフにし、ClustIceとgenice-coreで水の配置にしてみる。

g2 = nx.Graph([(i, j) for i, j in pl.pairs_iter(atomr, 0.33, cell, distance=False)])
# for i in range(atomr.shape):
#     g2[i]["pos"] = atomr[i]

# set orientations of the hydrogen bonds.
# if pos is given, the net dipole moment is minimized.
dg2 = genice_core.ice_graph(
    g2, vertexPositions=atomr @ cell, dipoleOptimizationCycles=1000
)
# dg = ice_graph(g)

# get the unique id for the graph
# id = serialize(dg)
# print(id)

# put water molecules
gro = gromacs.render(
    dg2,
    atomr @ cell,
    water_model=spce,
    cell_matrix=cell,
    # shift=np.array([0.5, 0.5, 0.5]),
)
with open("d-unitcell.gro", "w") as f:
    f.write(gro)

# show
view = py3Dmol.view()
view.addModel(gro, "gro")
view.setStyle({"stick": {}})  #: {"radius": 0.75}})
view.addUnitCell()
view.zoomTo()
view.show()
view.png()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [51]:
# GenIceのlatticeモジュールに落としこむ。
atomr.shape, cell

((112, 3),
 array([[2.2, 0. , 0. ],
        [0. , 1.7, 0. ],
        [0. , 0. , 2.8]]))

In [52]:
for x, y, z in atomr:
    print(f"{x:.3f} {y:.3f} {z:.3f}")

1.000 0.250 0.128
1.000 0.093 0.951
0.910 0.000 1.000
0.000 0.040 0.115
0.090 0.001 1.000
0.000 0.434 0.958
0.090 0.490 0.999
1.000 0.461 0.116
0.910 0.489 0.999
0.179 0.332 0.144
0.179 0.169 0.143
0.110 0.168 0.226
0.109 0.331 0.227
0.821 0.331 0.144
0.821 0.168 0.143
0.891 0.168 0.226
0.891 0.331 0.226
0.000 0.250 0.980
0.107 0.102 0.079
0.106 0.399 0.080
0.080 0.251 0.054
0.893 0.398 0.080
0.893 0.102 0.079
0.920 0.250 0.054
1.000 0.369 0.193
0.000 0.130 0.193
1.000 0.583 0.047
0.000 0.917 0.046
0.286 0.331 0.221
0.286 0.168 0.221
0.217 0.168 0.304
0.217 0.331 0.304
0.715 0.331 0.208
0.715 0.168 0.209
0.786 0.168 0.281
0.786 0.331 0.282
1.000 0.750 0.872
0.000 0.540 0.885
1.000 0.961 0.884
0.179 0.832 0.856
0.179 0.669 0.857
0.110 0.668 0.774
0.109 0.831 0.773
0.821 0.831 0.856
0.821 0.668 0.857
0.891 0.668 0.774
0.891 0.831 0.774
0.000 0.750 0.020
0.107 0.602 0.921
0.106 0.899 0.920
0.080 0.751 0.946
0.893 0.898 0.920
0.893 0.602 0.921
0.920 0.750 0.946
1.000 0.869 0.807
0.000 0.63

In [53]:
for i, j in g2.edges():
    print(i, j)

3 22
3 27
3 25
3 18
22 14
22 2
22 23
27 47
27 2
27 4
18 4
18 20
18 10
4 1
4 49
25 15
25 0
25 11
20 0
20 17
20 19
10 11
10 9
10 29
15 14
15 34
15 16
0 23
0 24
11 12
11 30
12 24
12 9
12 31
30 29
30 31
30 100
1 38
1 17
1 2
49 50
49 38
49 39
19 7
19 6
19 9
7 21
7 24
7 26
17 5
17 23
6 48
6 26
6 5
9 28
24 16
31 28
31 99
5 8
5 37
8 21
8 26
8 52
37 52
37 48
37 55
52 44
52 53
48 40
48 50
47 53
47 50
47 26
53 36
53 51
50 36
55 36
55 45
55 41
36 54
45 62
45 44
45 46
41 42
41 40
41 58
42 54
42 39
42 59
40 39
40 57
2 51
54 38
54 46
39 56
38 51
29 28
29 101
28 102
101 100
101 102
101 111
100 99
100 108
99 102
99 107
108 72
108 109
108 93
102 110
107 94
107 66
107 109
58 57
58 59
58 78
57 56
57 79
59 56
59 77
78 77
78 79
78 86
77 80
77 85
79 80
79 89
56 80
80 88
111 92
111 93
111 97
72 85
72 90
72 69
109 92
109 103
93 69
93 104
85 87
85 71
90 71
90 103
90 70
92 106
92 110
110 94
110 98
94 65
94 105
98 97
98 95
98 32
106 104
106 103
106 105
66 86
66 91
66 65
103 91
65 68
65 81
86 67
86 87
87 64
87 81
